In [24]:
!gdown 1WYynCgt3B4UZ7LVCGmLwvdEOrutGSVTt
!unzip -o MovieLens.zip

Downloading...
From: https://drive.google.com/uc?id=1WYynCgt3B4UZ7LVCGmLwvdEOrutGSVTt
To: /content/MovieLens.zip
100% 43.2M/43.2M [00:00<00:00, 158MB/s]
Archive:  MovieLens.zip
  inflating: movies.csv              
  inflating: ratings_df_sample_2.csv  


In [25]:
import pandas as pd
import random
import numpy as np

pd.options.display.max_colwidth = 2000
pd.options.display.float_format = '{:.2f}'.format

In [26]:
ratings = pd.read_csv('ratings_df_sample_2.csv')
movies = pd.read_csv('movies.csv')

In [27]:
df=(
    ratings
    .merge(movies, on='movieId')
)

In [28]:
df.shape

(6040099, 6)

In [29]:
%%time
movies_values = df['movieId'].unique()

df['movieId'] = df['movieId'].apply(lambda f: np.where(movies_values == f)[0][0])

CPU times: user 42.4 s, sys: 735 ms, total: 43.1 s
Wall time: 43.4 s


In [30]:
%%time
users_values = df['userId'].unique()

df['userId'] = df['userId'].apply(lambda f: np.where(users_values == f)[0][0])

CPU times: user 1min 54s, sys: 542 ms, total: 1min 54s
Wall time: 1min 55s


In [31]:
df['movie_viewed'] = 1

In [32]:
df.info(null_counts=True)

<ipython-input-32-acd3301a9435>:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  df.info(null_counts=True)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6040099 entries, 0 to 6040098
Data columns (total 7 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   userId        6040099 non-null  int64  
 1   movieId       6040099 non-null  int64  
 2   rating        6040099 non-null  float64
 3   timestamp     6040099 non-null  int64  
 4   title         6040099 non-null  object 
 5   genres        6040099 non-null  object 
 6   movie_viewed  6040099 non-null  int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 368.7+ MB


In [33]:
df.describe()

,userId,movieId,rating,timestamp,movie_viewed
count,6040099.00,6040099.00,6040099.00,6040099.00,6040099.00
mean,9115.02,422.90,3.55,1115774334.98,1.00
std,5563.83,283.85,1.00,135843321.30,0.00
min,0.00,0.00,0.50,824835410.00,1.00
25%,4365.00,165.00,3.00,995660158.00,1.00
50%,8679.00,415.00,4.00,1111706240.00,1.00
75%,13758.00,646.00,4.00,1213151458.50,1.00
max,19999.00,999.00,5.00,1427780469.00,1.00


In [34]:
df.sample(5)

,userId,movieId,rating,timestamp,title,genres,movie_viewed
4231391,6125,594,4.00,1086536548,Shrek 2 (2004),Adventure|Animation|Children|Comedy|Musical|Romance,1
1300330,153,143,3.50,1128619438,Die Hard: With a Vengeance (1995),Action|Crime|Thriller,1
5142523,11299,774,3.50,1146173053,White Men Can't Jump (1992),Comedy|Drama,1
637311,12003,61,3.00,1154348290,From Dusk Till Dawn (1996),Action|Comedy|Horror|Thriller,1
5073016,2351,757,3.00,1298849895,Wayne's World 2 (1993),Comedy,1


#Генерация нулей

In [35]:
uids = df['userId'].unique()
mids = df['movieId'].unique()

генерируем случайные пары

In [36]:
%%time
new_pairs = set()
for i in range(df.shape[0]*2):
  new_pairs.add((random.choice(uids), random.choice(mids)))

CPU times: user 30.8 s, sys: 378 ms, total: 31.2 s
Wall time: 31.4 s


сохраняем исходные пары ввиде множества

In [37]:
%%time
pairs = df[["userId", "movieId"]].values
pairs = set(tuple(i) for i in pairs)

CPU times: user 11.9 s, sys: 325 ms, total: 12.2 s
Wall time: 13.4 s


создаем новый датафрейм на основе разности множеств

In [38]:
%%time
pairs_df = pd.DataFrame(list(new_pairs.difference(pairs))[:df.shape[0]], columns=["userId", "movieId"])

CPU times: user 49.6 s, sys: 654 ms, total: 50.3 s
Wall time: 50.3 s


объединяем датафреймы

In [39]:
df_new = pd.concat([df,pairs_df], ignore_index=True)

In [40]:
df_new

,userId,movieId,rating,timestamp,title,genres,movie_viewed
0,0,0,3.00,974918176.00,Jumanji (1995),Adventure|Children|Fantasy,1.00
1,1,0,3.50,1112061358.00,Jumanji (1995),Adventure|Children|Fantasy,1.00
2,2,0,2.00,1132728068.00,Jumanji (1995),Adventure|Children|Fantasy,1.00
3,3,0,2.00,1134476330.00,Jumanji (1995),Adventure|Children|Fantasy,1.00
4,4,0,3.00,1283448701.00,Jumanji (1995),Adventure|Children|Fantasy,1.00
...,...,...,...,...,...,...,...
12080193,11132,870,NaN,NaN,NaN,NaN,NaN
12080194,13804,589,NaN,NaN,NaN,NaN,NaN
12080195,11030,593,NaN,NaN,NaN,NaN,NaN
12080196,19840,520,NaN,NaN,NaN,NaN,NaN


заполняем пропуски нулями

In [41]:
df_new = df_new.fillna(0)

избавляем от дубликатов

In [42]:
df_new.drop_duplicates(ignore_index=True)

,userId,movieId,rating,timestamp,title,genres,movie_viewed
0,0,0,3.00,974918176.00,Jumanji (1995),Adventure|Children|Fantasy,1.00
1,1,0,3.50,1112061358.00,Jumanji (1995),Adventure|Children|Fantasy,1.00
2,2,0,2.00,1132728068.00,Jumanji (1995),Adventure|Children|Fantasy,1.00
3,3,0,2.00,1134476330.00,Jumanji (1995),Adventure|Children|Fantasy,1.00
4,4,0,3.00,1283448701.00,Jumanji (1995),Adventure|Children|Fantasy,1.00
...,...,...,...,...,...,...,...
12080193,11132,870,0.00,0.00,0,0,0.00
12080194,13804,589,0.00,0.00,0,0,0.00
12080195,11030,593,0.00,0.00,0,0,0.00
12080196,19840,520,0.00,0.00,0,0,0.00


Дубликатов нет, сохраняем предобработанный датафрейм в файл

In [48]:
df_new.to_csv('data.csv', index=False)

In [49]:
df_new.describe()

,userId,movieId,rating,timestamp,movie_viewed
count,12080198.00,12080198.00,12080198.00,12080198.00,12080198.00
mean,9748.79,477.77,1.78,557887167.49,0.50
std,5728.84,289.40,1.91,566096123.13,0.50
min,0.00,0.00,0.00,0.00,0.00
25%,4799.00,224.00,0.00,0.00,0.00
50%,9593.00,472.00,0.25,412417705.00,0.50
75%,14670.00,724.00,4.00,1111706236.00,1.00
max,19999.00,999.00,5.00,1427780469.00,1.00


#Проверяем сохранненный файл

In [50]:
!gdown 1nPNY-p806LIL68kTok2hmkcQ8taTs9Bg

Downloading...
From: https://drive.google.com/uc?id=1nPNY-p806LIL68kTok2hmkcQ8taTs9Bg
To: /content/data.csv
100% 618M/618M [00:06<00:00, 97.5MB/s]


In [51]:
data = pd.read_csv('data.csv')

<ipython-input-51-71e55846462c>:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data.csv')


In [53]:
data.describe()

,userId,movieId,rating,timestamp,movie_viewed
count,12080198.00,12080198.00,12080198.00,12080198.00,12080198.00
mean,9748.79,477.77,1.78,557887167.49,0.50
std,5728.84,289.40,1.91,566096123.13,0.50
min,0.00,0.00,0.00,0.00,0.00
25%,4799.00,224.00,0.00,0.00,0.00
50%,9593.00,472.00,0.25,412417705.00,0.50
75%,14670.00,724.00,4.00,1111706236.00,1.00
max,19999.00,999.00,5.00,1427780469.00,1.00
